# Evaluate pydantic, the new validation library for DataClass

[pydantic](https://pydantic-docs.helpmanual.io/)

In [23]:
import typing as T
from pydantic import BaseModel, Field, validator

## PyCharm Support

If using pydantic, can PyCharm give:

- show argument name and type hint? (CMD + P)
- auto complete argument name? (just type in)
- show problems about typing mismatch, missing argument? (CMD + 6)

**Conclusion**

- YES it supports all, if you need to install [pydantic](https://plugins.jetbrains.com/plugin/12861-pydantic/versions) plugin.

In [ ]:
class Base(BaseModel):
    id: int


class User(Base):
    name: str


class PaidUser(User):
    account: str


# move cursor in brackt and hit CMD + P to see hint
Base()
User()
PaidUser(id="invalid", name="alice", account="1234")

## Nested Schema

You may need to nest an object as an attribute. In class initialization, you may use::

```
order = Order(
    ...
    user=User(name="alice@example.com"),
    ...
)
```

But when it serializes to dict, can you convert it back from dict? like this::

```
order_data = {..., "user": {"name": "alice@example.com"}, ...}
order = Order(**order_data)
```

**Conclusion**

- YES, it can. pydantic automatically add "converter" function based on type hint. In attrs, you have to manually specify converter.

In [3]:
class User(BaseModel):
    email: str


class Item(BaseModel):
    name: str
    quantity: int


class Order(BaseModel):
    id: int
    user: User
    items: T.List[Item]

In [6]:
order = Order(
    id=1,
    user=User(email="alice@example.com"),
    items=[Item(name="apple", quantity=3), Item(name="Banana", quantity=6)],
)
order

Order(id=1, user=User(email='alice@example.com'), items=[Item(name='apple', quantity=3), Item(name='Banana', quantity=6)])

In [8]:
order_data = order.dict()
order_data

{'id': 1,
 'user': {'email': 'alice@example.com'},
 'items': [{'name': 'apple', 'quantity': 3},
  {'name': 'Banana', 'quantity': 6}]}

In [9]:
order1 = Order(**order_data)
order1

Order(id=1, user=User(email='alice@example.com'), items=[Item(name='apple', quantity=3), Item(name='Banana', quantity=6)])

## Validator is also Converter

在 attrs 中, [对象的初始化的步骤](https://www.attrs.org/en/stable/init.html#order-of-execution) 是:

- ``__attrs_pre_init__`` hook (if present on current class)
- For each attribute, in the order it was declared:
    - default factory
    - converter
    - all validators
- ``__attrs_post_init__`` hook (if present on current class)

也就是说 converter 是单独的一个步骤, 并且发生在 validators 之前.

而在 pydantic 中 validator 本身就具备了 converter 的功能, 而且你可以自行定义 converter 是在 validator 之前还是之后.

In [22]:
class User(BaseModel):
    email: str

    @validator("email")
    def check_email(cls, v):
        if "@" not in v:
            raise ValueError(f"there's no @ in email {v!r}!")
        if not v.endswith(".com"):
            v = v + ".com"
        return v


class Item(BaseModel):
    name: str
    quantity: int


class Order(BaseModel):
    id: int
    user: User
    items: T.List[Item]


order = Order(
    id=1,
    user=User(email="alice@example"),
    items=[Item(name="apple", quantity=3), Item(name="Banana", quantity=6)],
)
print(order)

order_data = order.dict()
print(order_data)

order1 = Order(**order_data)
print(order1)

id=1 user=User(email='alice@example.com') items=[Item(name='apple', quantity=3), Item(name='Banana', quantity=6)]
{'id': 1, 'user': {'email': 'alice@example.com'}, 'items': [{'name': 'apple', 'quantity': 3}, {'name': 'Banana', 'quantity': 6}]}
id=1 user=User(email='alice@example.com') items=[Item(name='apple', quantity=3), Item(name='Banana', quantity=6)]


In [29]:
class Document(BaseModel):
    id: str = Field(alias="_id")


doc = Document(_id="doc_1")
doc.dict(by_alias=True)

{'_id': 'doc_1'}

In [30]:
class Document(BaseModel):
    ID: str = Field()


doc = Document(ID="doc_1")
doc.dict()

{'ID': 'doc_1'}

## Optional


In [33]:
class Data(BaseModel):
    c1: T.Optional[int] = Field(default=None)
    c2: int = Field()
    c3: T.Optional[int] = Field(default=None)
    c4: int = Field()

In [34]:
data = Data(c2=2, c4=4)
data

Data(c1=None, c2=2, c3=None, c4=4)

In [36]:
data.dict(exclude_none=True)

{'c2': 2, 'c4': 4}